<a href="https://colab.research.google.com/github/kalebmes/CS376-Machine-Learning/blob/main/Building_FC_and_CNN_layers_using_pytorch_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Training a fully connected neural network model for classifying digits

In [91]:
#create a fully connected Neural Network
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [92]:
#set a device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [93]:
#hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [94]:
#load the data
train = datasets.MNIST("", train=True, download=True, transform=transforms.ToTensor())
trainset = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test = datasets.MNIST("", train=False, download=True, transform=transforms.ToTensor())
testset = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)


In [95]:
#initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [96]:
#train the network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(trainset):
    #if possible, train it in cuda
    data = data.to(device=device)
    targets = targets.to(device=device)

    #since the dataset has a form of (m, input_channels, height, width) -> reshape it into (m, c*h*w)
    data = data.reshape(data.shape[0], -1)

    #forward propagation
    scores = model(data)
    loss = criterion(scores, targets)

    #backward propagation
    optimizer.zero_grad()
    loss.backward()

    #gradient descent
    optimizer.step()


In [104]:
def check_accuracy(loader, model):
  if trainset.dataset.train:
    print("Checking the accuracy of the training data")
  else:
    print("Checking the accuracy of the test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for (x, y) in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      # x = x.reshape(x.shape[0], -1)
      scores = model(x)
      _, predictions = scores.max(dim=1)
      num_correct += (predictions == y).sum()
      num_samples += (predictions.size(dim=0))
    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f}')
  model.train()

In [98]:
check_accuracy(trainset, model)
check_accuracy(testset, model)

Checking the accuracy of the training data
Got 55899 / 60000 with accuracy 93.16
Checking the accuracy of the training data
Got 9299 / 10000 with accuracy 92.99


### Convolutional Neural Networks

In [101]:
#create a simple CNN
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels=8, kernel_size=3, stride=(1, 1), padding=(1, 1)) #same convolution
    self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=(1, 1), padding=(1,1))
    self.fc1 = nn.Linear(7*7*16, num_classes)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x


In [102]:
#some additional hyperparameters
in_channel = 1
cnn_model = CNN().to(device)

#initialize an loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)

#train the Network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(trainset):
    #get the data into cuda if possible
    data, targets = data.to(device=device), targets.to(device=device)

    #Let's put in the correct shape when we implement the forward propagation
    # data = data.reshape(data.shape[0], -1)

    #forward propagation
    scores = cnn_model(data)
    loss = criterion(scores, targets)

    #backward propagation
    optimizer.zero_grad()
    loss.backward()

    #gradient descent or adam step
    optimizer.step()

In [105]:
check_accuracy(trainset, cnn_model)
check_accuracy(testset, cnn_model)

Checking the accuracy of the training data
Got 57725 / 60000 with accuracy 96.21
Checking the accuracy of the training data
Got 9673 / 10000 with accuracy 96.73
